In [15]:
import json

# Load your dataset (JSON format)
with open('BaselineResults\government_schemes.json', 'r', encoding='utf-8') as file:
    dataset = json.load(file)
# Prepare your dataset for fine-tuning (input-output pairs)
train_dataset = []
# i=1
for item in dataset:
#     i+=1
#     print(item['Description'])
#     if(i==10):
#         break
    #policy_name = item["Policy name"]
    train_dataset.append(f"input: Tell me about {item['Policy name']}") 
    train_dataset.append(f"output: {item['Description']['details']}")
    train_dataset.append("<|endoftext|>")
    # train_dataset.append(f"input: What are the benefits of {item['Policy name']} policy?")
    # train_dataset.append(f"output: {item['Description']['benefits']}")
    # train_dataset.append("<|endoftext|>")
    # train_dataset.append(f"Who is eligible for {item['Policy name']} policy?")
    # train_dataset.append(f"output: {item['Description']['eligibility']}")
    # train_dataset.append("<|endoftext|>")
train_dataset = "".join(train_dataset)
print('done')
print(train_dataset)

# Write the train_dataset_text to a text file
with open('preprocessed_scheme_data.txt', 'w', encoding='utf-8') as outfile:
    outfile.write(train_dataset)

done
input: Tell me about National AIDS Control Organisation (NACO) Internship Programmeoutput: NACO initiated the Internship Programme in 2018 for young students who wish to engage with the Government. The internship programme envisages an opportunity for young students to get familiar with and understand the various dimensions of policy-making & implementation of the National AIDS Control Programme. It will serve as an exposure for the interns regarding the functioning of the Government of India. It will be mutually beneficial for the organization as well as students to have a structured internship programme under the aegis of NACO, Ministry of Health and Family Welfare. The internship programme would provide an excellent opportunity for budding scholars to familiarise themselves with the process of formulation of health policy and its implementation by the Government of India, with specific reference to the National HIV/AIDS and STD Control Programme.﻿The internship is offered on va

In [16]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

def fine_tune_gpt2(model_name, train_file, output_dir):
    # Load GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Load training dataset
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=128)

    # Create data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False)

    # Set training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=5,
        per_device_train_batch_size=4,
        save_steps=10_000,
        save_total_limit=2,
    )

    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()

    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

fine_tune_gpt2("gpt2", "preprocessed_scheme_data.txt", "model_output")

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
100%|██████████| 20/20 [02:21<00:00,  7.06s/

{'train_runtime': 141.1479, 'train_samples_per_second': 0.531, 'train_steps_per_second': 0.142, 'train_loss': 6.001426696777344, 'epoch': 5.0}


In [17]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "model_output"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print("Chatbot: ")
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))


sequence = input("User: ")
max_len = 150
generate_text(sequence, max_len)

Chatbot: 
Tell about Livestock Health and Diseases Control and the impact of Livestock Health and Diseases Control.

Dirty Diseases in Livestock Health Control have been studied extensively by veterinary specialists and they will be used to develop new, safer solutions for livestock diseases. These strategies include:

Maintain an open lab environment (in all settings) that would minimize infection

Encourage open lab environment by prohibiting infection of livestock, animals and plants

Prevent disease transmission in all settings

Maintain the open lab environment and maintain appropriate procedures

Ensure humane treatment of diseased and diseased animals in open lab environment (i.e. the establishment of a public-private clinic). Livestock Health Control


